![Image of Yaktocat](http://i.stack.imgur.com/aogj9.gif)
# Part1: Crawler, Store Server, Repository

In [1]:
# crawl webpages

import urllib2
import urlparse

def simplifyUrl(url):
    return urlparse.urlsplit(url).geturl()

def crawlWebpage(url):
    connection = urllib2.urlopen(url)
    page = connection.read()
    connection.close
    return page
testPageUrl = 'https://en.wikipedia.org/wiki/Lightweight_Java_Game_Library'
testPageId = 1234
testPage = crawlWebpage(testPageUrl)
print simplifyUrl(testPageUrl)

https://en.wikipedia.org/wiki/Lightweight_Java_Game_Library


In [2]:
# store pages in repository

import zlib
import urlparse
import struct

packetFormat = 'IxHI'
packetByteSize = 12

def checksum(url):
    import binascii
    return binascii.crc32(url) & 0xffffffff #this keeps the results consistent across all platforms

#packs a given webpage information into bytes
def packWebpage(webpage, url, docID):
    url = simplifyUrl(url)
    packet = struct.pack(packetFormat, docID, len(url), len(webpage)) + url + webpage
    return packet

#unpacks a given packet into webpage, url, and docID
def unpackWebpage(packet):
    info = struct.unpack(packetFormat, packet[:packetByteSize])#because 3 arguments
    docID, urlLength, docLength = info[:]
    url = packet[packetByteSize:packetByteSize + urlLength]
    webpage = packet[packetByteSize + urlLength:]
    return webpage, url, docID

#packs a webpage and saves it by docID. save name is subject to change in the future.
def storePage(webpage, url, docID):
    url = simplifyUrl(url)
    print("Struct Size: %i" %struct.calcsize(packetFormat))
    packet = packWebpage(webpage, url, docID)
    compressedPacket = zlib.compress(packet)
    output = open('poople\\page_repository\\' + urlparse.urlparse(url)[1] + '%i' % docID, 'w')
    output.write(compressedPacket)
    output.close()

packet = packWebpage(testPage, testPageUrl, testPageId)
storePage(testPage, testPageUrl, testPageId)

Struct Size: 12


# Indexer(Parsing part) and Lexicon

In [3]:
# parse and simplify page

from bs4 import BeautifulSoup
import time
import re
import string

def simplifyPage(page):
    before = time.clock()
    soup = BeautifulSoup(testPage, "lxml")
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    text = re.sub(r'\W+', ' ', text)
    after = time.clock()
    print("Simplify Page Time: %f" %(after-before))
    return text

def parseWordsInPage(simplifiedPage):
    before = time.clock()
    #words = re.split(r'\W+', simplifiedPage)
    words = simplifiedPage.split(' ')
    after = time.clock()
    print("Parse Time: %f" %(after-before))
    for word in words:
        if word == "":
            words.remove(word)
    return words


# test page simplification
simplifiedPage = simplifyPage(testPage)
wordList = parseWordsInPage(simplifiedPage)
for word in wordList:
    print word

Simplify Page Time: 0.135038
Parse Time: 0.000172
Lightweight
Java
Game
Library
Wikipedia
the
free
encyclopedia
Lightweight
Java
Game
Library
From
Wikipedia
the
free
encyclopedia
Jump
to
navigation
search
This
article
has
multiple
issues
Please
help
improve
it
or
discuss
these
issues
on
the
talk
page
Learn
how
and
when
to
remove
these
template
messages
The
topic
of
this
article
may
not
meet
Wikipedia
s
general
notability
guideline
Please
help
to
establish
notability
by
citing
reliable
secondary
sources
that
are
independent
of
the
topic
and
provide
significant
coverage
of
it
beyond
its
mere
trivial
mention
If
notability
cannot
be
established
the
article
is
likely
to
be
merged
redirected
or
deleted
Find
sources
Lightweight
Java
Game
Library
news
newspapers
books
scholar
JSTOR
free
images
March
2016
Learn
how
and
when
to
remove
this
template
message
This
article
relies
too
much
on
references
to
primary
sources
Please
improve
this
by
adding
secondary
or
tertiary
sources
August
2015
Learn
h

In [4]:
# TO DELETE

import sys
import time
import binascii

#The hash we are using. Can be changed without affecting the program
def wordHash(word):
    word = word.lower()
    return binascii.crc32(word) & 0xffffffff

#adds the word to the hashtable
#saves the word to the file so next time it will be loaded like normal
def addWordToLexicon(word, hashtable):
    word = word.lower()
    hashtable = addWordToHashtable(word, hashtable)
    saveNewWord(word)
    return hashtable

#adds word to given hashtable. Does not save to file. Hashes word
def addWordToHashtable(word, hashtable):
    word = word.lower()
    hashtable[wordHash(word)] = len(hashtable)
    return hashtable

#gets the wordID from the hashtable
def getWordId(word, hashtable):
    try:
        lowerWord = word.lower()
        return hashtable[wordHash(lowerWord)]
    except KeyError:
        print "Word not in lexicon: " + word
        return -1;

#saves the word to a file
def saveNewWord(word):
    word = word.lower()
    word = word + ","
    output = open('poople\\lexicon', 'a')
    output.write(word)
    output.close()
    
#loads the lexicon from file. creates the hashtable from lexicon
def loadLexicon():
    lexiconList = []
    stream = open('poople\\lexicon', 'r')
    lexicon = stream.read()
    stream.close()
    lexiconList = lexicon.split(",")
    hashtable = {}
    for word in lexiconList:
        if word != '':
            hashtable = addWordToHashtable(word, hashtable)
    return hashtable
    
wordHashtable = loadLexicon()
print len(wordHashtable)
addWordToHashtable("hello", wordHashtable)
for word in simplifiedPage.split(" "):
    addWordToHashtable(word, wordHashtable)
print getWordId("hello", wordHashtable)
print len(wordHashtable)
print wordHashtable

1
1
696
{0L: 696, 1788938242L: 570, 3187900075L: 537, 3230498821L: 487, 3747223559L: 281, 835352588L: 428, 2902841359L: 359, 2260480018L: 516, 820668437L: 646, 1718319126L: 679, 1154021400L: 582, 1724430365L: 544, 1423829025L: 578, 64952355L: 0, 1843675174L: 279, 3358384175L: 669, 1964050481L: 254, 2568837171L: 556, 1762699316L: 255, 1574674497L: 610, 1060745282L: 393, 4052537413L: 542, 689295432L: 403, 666529867L: 637, 1280163916L: 419, 2838417488L: 357, 3110191185L: 226, 2252471652L: 374, 460212315L: 335, 146714717L: 539, 2897602656L: 458, 711391330L: 568, 1630611555L: 158, 395401319L: 651, 306016363L: 463, 1972041839L: 383, 3218944360L: 625, 1673277559L: 78, 1917560954L: 71, 2508312701L: 92, 3065852031L: 626, 1437575297L: 555, 2796017799L: 479, 3265925258L: 504, 2297403532L: 399, 1458215053L: 667, 1997877400L: 251, 376615066L: 46, 4097009823L: 660, 2458417313L: 351, 1877817509L: 353, 4038908070L: 586, 2854008999L: 482, 2223210669L: 585, 432980143L: 493, 1638783154L: 303, 2892185779L

# Anchors, URL Resolver, Links, Doc Index, URL Server

# Indexer(rest) Barrels, Sorter, PageRank

In [5]:
#handle (en/de)coding hits

def encodePlainHit(cap, imp, pos):
    result = 0
    if(cap):
        result +=   0b1000000000000000 #2^16
    result += imp << 12
    result += pos & 0b0000111111111111
    return struct.pack("H", result)
def encodeFancyHit(cap, ftype, pos):
    result = 0
    if(cap):
        result +=   0b1000000000000000 #2^16
    result += 0b0111000000000000 #fancy hits are always 7
    result += ftype << 8
    return struct.pack("H", result)
def encodeAnchorHit(cap, ahash, pos):
    result = 0b0111000000000000
    if(cap):
        result +=   0b1000000000000000 #2^16
    result += pos
    result += ahash << 4
    return struct.pack("H", result)

def decodeHit(packedHit):
    hit = struct.unpack("H", packedHit)[0]
    result = []
    result.append(hit >> 15)
    imp = (hit >> 12) & 0b111
    result.append(imp)
    if(imp != 7):
        result.append(hit & 0b0000111111111111)
    else: #fancy hit
        fancyType = (hit >> 8) & 0b1111
        result.append(fancyType)
        if(fancyType == 0): #anchor hit
            result.append(hit >> 4 & 0b1111)
            result.append(hit & 0b1111)
        else:
            result.append(hit & 0b11111111)
    return result

hit = encodePlainHit(False, 4, 0b001100000111)
fhit = encodeFancyHit(True, 0b0111, 0b10000000)
ahit = encodeAnchorHit(True, 0b1000, 0b1000)
print(decodeHit(hit))
print(decodeHit(fhit))
print(decodeHit(ahit))

[0, 4, 775]
[1, 7, 7, 0]
[1, 7, 0, 8, 8]


In [6]:
# start to put together forward indices
# hits[n] = (wordId, encodedHit)

#BARREL VARIABLES:
numBarrels = 64

def printBytes(string):
    print(string)
    print(':'.join(x.encode('hex') for x in string))

def seekNextDocIdFor(fileText, csid): # helper method for saveToForwardBarrels
    if(csid == len(fileText)):
        return -1
    csid += 4 # seek past current docId
    numHits = 1
    while(numHits != 0):
        numHits = struct.unpack("L", fileText[csid:csid+4])[0] & 0b11111111
        csid += 4+numHits*2 #fix this
    return csid

def saveToForwardBarrels(docId, hits):
    tempLex = {}
    for hit in hits:
        if(hit[0] in tempLex):
            tempLex[hit[0]] += hit[1]
        else:
            tempLex[hit[0]] = hit[1]

    barrels = []
    for i in xrange(numBarrels):
        barrels.append("")
    for key in tempLex.keys():
        if len(barrels[key >> 24]) == 0:
            barrels[key >> 24] = struct.pack("L", docId)
        barrels[key >> 24] += struct.pack("L", ((key&0b111111111111111111111111) << 8) + (len(tempLex[key])/2 & 0b11111111)) + tempLex[key]
            
    for i in xrange(len(barrels)):
        if(len(barrels[i])):
            entry = barrels[i] + struct.pack("L", 0) # null wordId with 0 hits
            try: # read the file
                readFile = open('poople\\for\\b%i' % i, 'r')
                fileText = readFile.read()
                printBytes(fileText)
                readFile.close()
            except IOError: # create the file if it's not there
                print("BARREL CREATED")
                fileText = entry
            else:
                sid = 0
                while True:
                    nsid = seekNextDocIdFor(fileText, sid)
                    if(nsid == -1):
                        print("ENTRY APPENDED")
                        fileText += entry
                        break
                    currentDocId = struct.unpack('L', fileText[sid:sid+4])[0]
                    if(currentDocId == docId):
                        print("ENTRY REPLACED")
                        fileText = fileText[0:sid] + entry + fileText[nsid:]
                        break
                    elif(currentDocId > docId):
                        print("ENTRY INSERTED")
                        fileText = fileText[0:sid] + entry + fileText[sid:]
                        break
                    sid = nsid
                    
            # write altered fileText back to file
            writeFile = open('poople\\for\\b%i' % i, 'w')
            writeFile.write(fileText)
            writeFile.close()

testHit = "AA"       #(wordId,                           hit)
testBarrelSaveData = [(0b101101001011010101101001011010, testHit),
                      (0b101101001011010101101001011010, testHit),
                      (0b101101001011010101101001011010, testHit),
                      (0b00011111111111111111111100001111, testHit),
                      (0b00111111111111111111111111111111, testHit)]
saveToForwardBarrels(3, testBarrelSaveData) # THIS WORKS NOW!

   ��AA       ��AA       ��AA       ��AA    
01:00:00:00:01:0f:ff:ff:41:41:00:00:00:00:02:00:00:00:01:0f:ff:ff:41:41:00:00:00:00:03:00:00:00:01:0f:ff:ff:41:41:00:00:00:00:04:00:00:00:01:0f:ff:ff:41:41:00:00:00:00
ENTRY REPLACED
   ZZ-AAAAAA       ZZ-AAAAAA       ZZ-AAAAAA       ZZ-AAAAAA    
01:00:00:00:03:5a:5a:2d:41:41:41:41:41:41:00:00:00:00:02:00:00:00:03:5a:5a:2d:41:41:41:41:41:41:00:00:00:00:03:00:00:00:03:5a:5a:2d:41:41:41:41:41:41:00:00:00:00:04:00:00:00:03:5a:5a:2d:41:41:41:41:41:41:00:00:00:00
ENTRY REPLACED
   ���AA       ���AA       ���AA       ���AA    
01:00:00:00:01:ff:ff:ff:41:41:00:00:00:00:02:00:00:00:01:ff:ff:ff:41:41:00:00:00:00:03:00:00:00:01:ff:ff:ff:41:41:00:00:00:00:04:00:00:00:01:ff:ff:ff:41:41:00:00:00:00
ENTRY REPLACED


In [7]:
# create backwards indices

def seekNextDocIdInv(fileText, sid):
    if(sid == len(fileText)):
        return -1
    numHits = struct.unpack("L", fileText[sid:sid+4])[0] & 0b11111
    return sid+4+numHits*2

def saveInvertedIndex(barrelNum):
    try:
        readFile = open('poople\\for\\b%i' % barrelNum, 'r')
        fileText = readFile.read()
        readFile.close()
    except IOError:
        return
    
    # convert fileText to wordHits
    sid = 0
    wordHits = {}
    while True:
        nsid = seekNextDocIdFor(fileText, sid)
        if(nsid == -1):
            break
        entry = fileText[sid:nsid]
        docId = struct.unpack("L", entry[0:4])[0]
        masked = struct.unpack("L", entry[4:8])[0]
        wordId = masked >> 8
        numHits = masked & 0b11111111
        hits = entry[8:8+numHits*2]
        if(not wordId in wordHits.keys()):
            wordHits[wordId] = {}
        if(not docId in wordHits[wordId]):
            wordHits[wordId][docId] = hits
        else:
            wordHits[wordId][docId] += hits
        sid = nsid
        
    print("IT GOT THIS FAR")
    print(wordHits)
        
    # save hit lists to their inv files
    for wordId in wordHits.keys():
        
        # load and alter fileText
        try:
            readFile = open('poople\\inv\\b%i' % wordId, 'r')
            fileText = readFile.read()
            readFile.close()
            print("FILE READ SUCCESSFULLY")
        except IOError:
            print("EMPTY FILE")
            fileText = struct.pack("L", 0)
        
        numDocs = struct.unpack("L", fileText[0:4])[0]
        
        for docId in wordHits[wordId].keys():
            printBytes(fileText)
            print(docId)
            sid = 4 # numDocs is the first 4 chars
            entry = struct.pack("L", (docId << 5) + (len(wordHits[wordId][docId])/2)) + wordHits[wordId][docId]
            while True:
                nsid = seekNextDocIdInv(fileText, sid)
                if(nsid == -1):
                    print("ENTRY APPENDED")
                    fileText = struct.pack("L", numDocs+1) + fileText[4:] + entry
                    break
                currentDocId = struct.unpack("L", fileText[sid:sid+4])[0] >> 5
                if(currentDocId == docId):
                    print("ENTRY REPLACED")
                    fileText = fileText[0:sid] + entry + fileText[nsid:]
                    break
                elif(currentDocId > docId):
                    print("ENTRY INSERTED")
                    fileText = struct.pack("L", numDocs+1) + fileText[4:sid] + entry + fileText[sid:]
                    break
                sid = nsid
        
        # write altered fileText
        writeFile = open('poople\\inv\\b%i' % wordId, "w")
        writeFile.write(fileText)
        writeFile.close()
        
saveInvertedIndex(45)

992
IT GOT THIS FAR
{2972250: {1: 'AAAAAA', 2: 'AAAAAA', 3: 'AAAAAA', 4: 'AAAAAA'}}
FILE READ SUCCESSFULLY
   #}  AAAAAAC}  CCCCCCc}  EEEEEE�}  DDDDDD
01:00:00:00:23:7d:00:00:41:41:41:41:41:41:43:7d:00:00:43:43:43:43:43:43:63:7d:00:00:45:45:45:45:45:45:83:7d:00:00:44:44:44:44:44:44
1
ENTRY INSERTED
   #   AAAAAA#}  AAAAAAC}  CCCCCCc}  EEEEEE�}  DDDDDD
02:00:00:00:23:00:00:00:41:41:41:41:41:41:23:7d:00:00:41:41:41:41:41:41:43:7d:00:00:43:43:43:43:43:43:63:7d:00:00:45:45:45:45:45:45:83:7d:00:00:44:44:44:44:44:44
2
ENTRY INSERTED
   #   AAAAAAC   AAAAAA#}  AAAAAAC}  CCCCCCc}  EEEEEE�}  DDDDDD
02:00:00:00:23:00:00:00:41:41:41:41:41:41:43:00:00:00:41:41:41:41:41:41:23:7d:00:00:41:41:41:41:41:41:43:7d:00:00:43:43:43:43:43:43:63:7d:00:00:45:45:45:45:45:45:83:7d:00:00:44:44:44:44:44:44
3
ENTRY INSERTED
   #   AAAAAAC   AAAAAAc   AAAAAA#}  AAAAAAC}  CCCCCCc}  EEEEEE�}  DDDDDD
02:00:00:00:23:00:00:00:41:41:41:41:41:41:43:00:00:00:41:41:41:41:41:41:63:00:00:00:41:41:41:41:41:41:23:7d:00:00:41

In [8]:
# read from invertedIndices

def loadInvertedIndex(wordId):
    
    # read file
    try:
        readFile = open("poople\\inv\\%i" % wordId, "r")
        content = readFile.read()
        readFile.close()
    except IOError:
        print("WORD NOT IN BACKWARDS INDEXES")
        return {}
    # load hit lists
    hits = {}
    numDocs = struct.unpack("L", content[0:4])[0]
    currentSeekIndex = 4
    for i in xrange(numDocs):
        temp = struct.unpack("L", content[currentSeekIndex:currentSeekIndex+4])[0]
        docId = temp >> 5
        numHits = temp & 0b11111
        hits[docId] = content[currentSeekIndex+4:currentSeekIndex+4+2*numHits]
        currentSeekIndex += 4+numHits*2
    return hits

loadInvertedIndex(0)

WORD NOT IN BACKWARDS INDEXES


{}

# Ranking System, Searcher